In [40]:
#pip install pandas


In [12]:
#pip install fuzzywuzzy

In [4]:
from fuzzywuzzy import process
str2Match = "skill"
strOptions = ["skill level 1", "primary skills", "secondary skills", "skill set 2", "Resource Work City", "Account Classification"]
Ratios = process.extract(str2Match,strOptions)
print(Ratios)
# You can also select the string with the highest matching percentage
highest = process.extractOne(str2Match,strOptions)
print(highest)

[('skill level 1', 90), ('primary skills', 90), ('secondary skills', 90), ('skill set 2', 90), ('Resource Work City', 36)]
('skill level 1', 90)


C:\ProgramData\Miniconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
import pandas as pd
df =pd.read_csv("GBSSampleData2.csv",header=0,encoding="ISO-8859-1")
ls= df.columns
list(df.columns)

['Contractor Indicator',
 'Name',
 'Serial',
 'CNUM',
 'ISO Country Code',
 'Company',
 'Country + Serial',
 'Currently Active',
 'Serial + Country + Company',
 'JRSS',
 'Complex Program Manager Flag',
 'JRSS Short List',
 'Direct JRSS',
 'Labor Group',
 'Labor Pool',
 'RSP',
 'Staffing Reviewer',
 'Availability Date',
 'Available Hours',
 'System Calculated Availability Date',
 'Geography Type',
 'Geography',
 'Market',
 'Market Region',
 'Country',
 'Sub-Country',
 'BMDiv',
 'Division',
 'CIC Center Group',
 'CIC Center',
 'Service',
 'Practice',
 'Service Area',
 'Service Component',
 'Team',
 'Sector',
 'Industry',
 'EBU Code',
 'Extension Date',
 'FA Code',
 'Charge Group Code',
 'Cost Group Code',
 'LERU ID',
 'Fin Dept ID',
 'Dept Major',
 'Dept Minor',
 'BP Dept',
 'Dept Cat Code',
 'HR Dept',
 'Department Name',
 'EMF Source',
 'EMF Status',
 'EMF Longevity Code',
 'Manager',
 'Functional Manager',
 'Manager Flag',
 'Band',
 'ISV (Vendor)',
 'Travel Preference',
 'Weeks to Ava

## Deploying a Decision Optimization model with Watson Machine Learning



In [41]:
#pip install docplex

In [15]:
#pip install botocore

In [17]:
import types
import pandas as pd
from botocore.client import Config
from datetime import datetime, timedelta
#import ibm_boto3
import sys
import docplex.mp
from docplex.mp.model import Model
from io import StringIO
import json
from pandas import DataFrame

def __iter__(self): return 0

   
# load csv and get the relevant columns
df = pd.read_csv("data.csv",header=0,encoding="ISO-8859-1")
df.head()
df1=df[["Employee ID","Primary Skill Set","Skill 2","Availability Date","Employee Country","Service City"]]

df1.head()

,Employee ID,Primary Skill Set,Skill 2,Availability Date,Employee Country,Service City
0,00450P744,SAP.BOBJ,Python,30-07-2018,India,Jaipur
1,00818F744,SAP.ABAP,Python,31-12-2018,India,BANGALORE
2,00554K744,JDE,Python,1/1/2020,India,NOIDA
3,00696V744,Java.core,Python,30-07-2018,India,GURGAON
4,00340B744,Custom Application,Python,31-12-2018,India,BANGALORE


In [18]:
df1["Primary Skill Set"].value_counts()

SAP.FIN.FI                           12
SAP.ABAP                             10
Oracle Database                      10
Java.core                             5
Technincal Requirement                5
Custom Application                    5
Test Automation                       5
Java.Spring & Hibernate               5
SAP.SCM.SD                            5
COBOL                                 5
Application Maintenance & Support     5
SAP.BASIS                             5
c#.NET                                5
JDE                                   5
Oracle Application                    5
Custom Development                    5
SAP.BOBJ                              5
Name: Primary Skill Set, dtype: int64

In [19]:
# hard code the external inputs for now
required_no_of_employees = 14
required_location = "NOIDA"
required_skills = "Oracle Database"
required_in_days = 60
assignment_cost = 0
training_cost = 1000
relocation_cost = 2000



In [21]:

#employee_serial = df1["Serial"]
employee_cnum = df1 ["Employee ID"]
employee_availability_date = df1["Availability Date"]
#employee_notesID = df1["Practitioner Notes ID"]
employee_primarySkill = df1["Primary Skill Set"]


#target date in input
# calculate intermediaries
target_date =  datetime. today() + timedelta(required_in_days)
target_date_minus_30 = target_date - timedelta(30)
target_date_plus_30 = target_date + timedelta (30)
#print(target_date_minus_30)
#print(target_date_plus_30)


In [22]:
# copy only rows where emp availability date is earlier than target date  
df1 = df1.dropna(how = 'all') 
# take care of formatting the orginial Availability Date before copying. Hardcoding format for now
df_filtered = df1[df1["Availability Date"] <= target_date.strftime("%d-%m-%Y")]


df_filtered.head()

,Employee ID,Primary Skill Set,Skill 2,Availability Date,Employee Country,Service City
0,00450P744,SAP.BOBJ,Python,30-07-2018,India,Jaipur
2,00554K744,JDE,Python,1/1/2020,India,NOIDA
3,00696V744,Java.core,Python,30-07-2018,India,GURGAON
5,00538P744,SAP.ABAP,Python,1/1/2020,India,NOIDA
6,00130H744,COBOL,Python,30-07-2018,India,GURGAON


In [23]:
df_filtered['Primary Skill Set'].value_counts()

SAP.FIN.FI                           6
SAP.SCM.SD                           4
SAP.BASIS                            4
SAP.ABAP                             4
COBOL                                4
Java.Spring & Hibernate              4
Oracle Application                   3
c#.NET                               3
Application Maintenance & Support    3
Java.core                            3
Oracle Database                      3
Technincal Requirement               3
Custom Application                   2
Custom Development                   2
JDE                                  2
SAP.BOBJ                             2
Test Automation                      1
Name: Primary Skill Set, dtype: int64

In [25]:
employee_work_loc= "Service City"
employee_primary_skillset= "Primary Skill Set"

# add a new column cost 
# add relocation cost if available employee is based out of another work location
# add training cost if the available employee does not have the required skills
df_filtered['Relocation Cost'] = [relocation_cost if x != required_location.upper() else 0 for x in df_filtered['Service City']]
df_filtered['Training Cost'] = [training_cost if (y.find(required_skills)==-1) else 0 for y in df_filtered['Primary Skill Set']]
df_filtered['Total Cost'] = df_filtered['Relocation Cost'] + df_filtered['Training Cost']

df1=df_filtered[["Employee ID","Primary Skill Set","Service City","Relocation Cost","Training Cost","Total Cost"]]

#assignments= mdl.binary_var_dict(keys1=employee_notesID)

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [26]:
#project.save_data(data=df1.to_csv(index=False),file_name='GBS.csv',overwrite=True)

In [27]:
df1.head()


,Employee ID,Primary Skill Set,Service City,Relocation Cost,Training Cost,Total Cost
0,00450P744,SAP.BOBJ,Jaipur,2000,1000,3000
2,00554K744,JDE,NOIDA,0,1000,1000
3,00696V744,Java.core,GURGAON,2000,1000,3000
5,00538P744,SAP.ABAP,NOIDA,0,1000,1000
6,00130H744,COBOL,GURGAON,2000,1000,3000


In [28]:
#df2['Need']=['Training' if x != 0 else 0 for x in df_filtered['Training Cost']]
#df2.head()
def func(x,y):
    if x!=0 or y!=0:
        if x!=0 and y!=0:
            return 'Relocation_Training'
        elif x!=0:
            return "Training"
        elif y!=0:
            return "Relocation"
    else:
        return "No Relocation and Training"
        
df2=df1.copy()       
df2['Need']=df2.apply(lambda x: func(x['Training Cost'],x['Relocation Cost']),axis=1)

In [29]:

df2=df2.reset_index(drop= True)
df2.head()

,Employee ID,Primary Skill Set,Service City,Relocation Cost,Training Cost,Total Cost,Need
0,00450P744,SAP.BOBJ,Jaipur,2000,1000,3000,Relocation_Training
1,00554K744,JDE,NOIDA,0,1000,1000,Training
2,00696V744,Java.core,GURGAON,2000,1000,3000,Relocation_Training
3,00538P744,SAP.ABAP,NOIDA,0,1000,1000,Training
4,00130H744,COBOL,GURGAON,2000,1000,3000,Relocation_Training



# Cplex Modeling

In [30]:
Training_req_count=len(df2[df2['Need']=='Training'])
Relocation_req_count = len(df2[df2['Need']=='Relocation'])
T_R_req_count= len(df2[df2['Need']=='Relocation_Training'])
N_T_R_req_count=len(df2[df2['Need']=='No Relocation and Training'])

In [31]:
Total_avialable_candidate =Training_req_count+ Relocation_req_count + T_R_req_count + N_T_R_req_count

In [32]:
print('Training',Training_req_count)
print('Relocation',Relocation_req_count)
print('T_R_req_counr',T_R_req_count)
print('N_T_R_req_counr',N_T_R_req_count)

Training 16
Relocation 1
T_R_req_counr 34
N_T_R_req_counr 2


In [33]:
required_no_of_employees

14

# Docplex Modeling

In [34]:
def docplex_modeling(Training_req_count,Relocation_req_count,T_R_req_count,N_T_R_req_count,required_no_of_employees,Total_avialable_candidate,need_extra_solution):
    
    mdl = Model(name="Assignments")

    Training = mdl.integer_var(name="Training")
    
    Relocation = mdl.integer_var(name ="Relocation")

    Relocation_Training = mdl.integer_var(name="Relocation_Training")



    N_Train_Relocation = mdl.integer_var(name="No Relocation and Training")


    # Extra variable when required no of employees are greater then avialable emp

    variables= ['Training','Relocation','Relocation_Training''N_Train_Relocation']
    
    names=['Training','Relocation','Relocation_Training','No Relocation and Training']

    no_of_var= len(variables)

    constraints= ['Training','Relocation_Training','Relocation','N_Train_Relocation']
    
    sol=[]  # extra solutions
    
    objs =[]  # there corresponding costs


    #Adding constraints
    
    # assign Training variable to only those who need training
    mdl.add_constraint(Training <= Training_req_count)  
    
    # assign Relocation variable to only those who need Relocation
    mdl.add_constraint(Relocation <= Relocation_req_count)
    
    
    mdl.add_constraint(N_Train_Relocation<=N_T_R_req_count)
    
    
    mdl.add_constraint(Relocation_Training <= T_R_req_count)
    
        
    
    ct_no_of_req =mdl.add_constraint((Relocation + Relocation_Training + Training + N_Train_Relocation)==required_no_of_employees)
    
    
    
    #Defining total cost
    tot_cost=10000* Training + 20000 * Relocation + 30000 * Relocation_Training
    
    
    
    mdl.add_kpi(tot_cost,'tot_cost') #add kpi
    
    
    #If two more solutions are required
    
    
    if need_extra_solution==2:
        
        #Defining extra constraints
        if Relocation_req_count > 0.05 * required_no_of_employees:
            mdl.add_constraint(Relocation>=0.05 * required_no_of_employees)
            
        if Training_req_count >=0.10 * required_no_of_employees:
            mdl.add_constraint(Training >=0.10 * required_no_of_employees)
        
        if T_R_req_count>=0.05 * required_no_of_employees:
            mdl.add_constraint(Relocation_Training >= 0.05 * T_R_req_count)
        
        mdl.minimize(tot_cost)
        
        mdl.solve()
        ls=[]
        #print('SECOND SOLUTION')
        #appending cost
        objs.append(mdl.solution.get_objective_value())
        for i in range(no_of_var):
            
            value=mdl.get_var_by_index(i).solution_value
            if value!=0:
                print(names[i]+' :'+str(value))
            ls.append(value)
            #Selected_data= Selected_data.append(pd.DataFrame(df2[df2['Need']==names[i]].head(int(value))))
        
        
        sol.append(ls)  #appending solution
        #mdl.print_solution()
    if Relocation_req_count>= required_no_of_employees*0.10:
        mdl.add_constraint(Relocation>=required_no_of_employees*0.10)
        
    #mdl.add_constraint(Training >=0.10 * Training_req_count)
    #mdl.add_constraint(Relocation_Training >= 0.10 * T_R_req_count)
    mdl.minimize(tot_cost)
    
    Selected_data = pd.DataFrame(columns=df2.columns)
#optimal solution
    print('Other solutions')
    if True:
        mdl.solve()
        #mdl.print_solution()
        ls=[]
        objs.append(mdl.solution.get_objective_value())
        for i in range(no_of_var):
            value=mdl.get_var_by_index(i).solution_value
            if value!=0:
                print(names[i]+' :'+str(value))
            ls.append(value)
            #Selected_data= Selected_data.append(pd.DataFrame(df2[df2['Need']==names[i]].head(int(value))))
        sol.append(ls)
    return sol,objs
#Selected_data.to_csv('Selected_Candidate.csv')
    #except:
     #   pass




    

#assert mdl.solve(), 
#mdl.solve() 
#mdl.print_solution()# Optimal solution
    

In [39]:
#pip install cplex

In [38]:
#Cplex Modeling
import warnings
warnings.simplefilter('ignore')

import cplex
from cplex.exceptions import CplexError
import sys
import os
import sys
import pandas as pd
#import types
import pandas as pd
#from botocore.client import Config
from datetime import datetime, timedelta
#import ibm_boto3
import sys
import docplex.mp
#from io import StringIO
import json
from pandas import DataFrame




# Defining The problem , constraints, objctives

def populatebyrow(prob):
    prob.objective.set_sense(prob.objective.sense.minimize)

    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj = my_obj, ub = my_ub, names = my_colnames,types =[prob.variables.type.integer] * 4)

    # can query variables like the following bounds and names:

    # lbs is a list of all the lower bounds
    lbs = prob.variables.get_lower_bounds()

    # ub1 is just the first lower bound
    #ub1 = prob.variables.get_upper_bounds(0)

    # names is ["x1", "x3"]
    names = prob.variables.get_names([0, 2])

    rows = [[["Training","Relocation","Relocation_Training","No_Relocation_and_Training"],[1.0, 1.0,1.0,1.0,1.0]]]

    prob.linear_constraints.add(lin_expr = rows, senses = my_sense,
                                rhs = my_rhs, names = my_rownames)

    


def lpex1():
    my_prob = cplex.Cplex()
    populatebyrow(my_prob)
    my_prob.solve()
    numrows = my_prob.linear_constraints.get_num()
    numcols = my_prob.variables.get_num()
        # solution.get_status() returns an integer code
    #print("Solution status = " , my_prob.solution.get_status(), ":")
    # the following line prints the corresponding string
    #print(my_prob.solution.status[my_prob.solution.get_status()])
    #print("Solution value  = ", my_prob.solution.get_objective_value())
    slack = my_prob.solution.get_linear_slacks()
    #pi    = my_prob.solution.get_dual_values()
    x     = my_prob.solution.get_values()
   # dj    = my_prob.solution.get_reduced_costs()
    #for i in range(numrows):
     #   print("Row %d:  Slack = %10f  Pi = %10f" % (i, slack[i], pi[i]))
    #for j in range(numcols):
     #   print("Column %d:  Value = %d" % (j, x[j]))

    #my_prob.write("x1")
    #my_prob.solve()
    
    #Setting parameters for solution pool
    my_prob.presolve =1
    
    # Set the capacity of solution pool to 10
    my_prob.parameters.mip.pool.capacity=10
    
    #Set the intensity of solution to most aggresive one , so that more solution can be obtained
    my_prob.parameters.mip.pool.intensity =4
   # my_prob.parameters.mip.pool.replace = 2

    #Set the node limit to a very high number
    my_prob.parameters.mip.limits.populate =  2100000000
    
    #set timelimit for serching to 10 seconds 
    my_prob.parameters.timelimit = 10
    
    #Getting all possible incumbent solution
    my_prob.populate_solution_pool()
    
    
    #iterating through all possible solution
    numsol = my_prob.solution.pool.get_num()
    
    print("The solution pool contains %d solutions." % numsol)
    
    meanobjval = my_prob.solution.pool.get_mean_objective_value()
    sol_pool = []
    objval=[]
    numsol = my_prob.solution.pool.get_num()
    
    #store solutions
    for i in range(numsol):
       
        objval.append(my_prob.solution.pool.get_objective_value(i))
        x_i = my_prob.solution.pool.get_values(i)
        nb_vars=len(x_i)
        sol = []
        for k in range(nb_vars):
            sol.append(x_i[k])
        sol_pool.append(sol)
        #my_prob.solution.pool.delete(0)
    #my_prob.parameters.tune_problem
    #Selected_cand(sol_pool)
    
    #print("pools =",sol_pool)
    return sol_pool, objval


#If the required no of emp are more then the the toatal no of employees avialable 
#Assign the all avialable and ret for Hire


#initialising the variable for hiring 
Need_To_Hire=0

#Checking Condition
if required_no_of_employees > Total_avialable_candidate:
    Need_To_Hire = required_no_of_employees-Total_avialable_candidate
    print(Need_To_Hire)
    #Assign all the avialable 
    required_no_of_employees= Total_avialable_candidate
    
# data common to all populateby functions

 
train_Relocation_cost= training_cost + relocation_cost
N_Train_Reloc=0

#my_obj      = [10000, 20000, 30000, 0]

my_obj      = [training_cost, relocation_cost, train_Relocation_cost, N_Train_Reloc]

#Setting Upperbounds of values of all the variables
my_ub       = [Training_req_count, Relocation_req_count, T_R_req_count,N_T_R_req_count ]
my_colnames = ["Training", "Relocation", "Relocation_Training","No_Relocation_and_Training"]

my_rhs      = [required_no_of_employees]
my_rownames = ["c1"]
my_sense    = "E"

#DataFrame for storing final candiates names
Selected_data=pd.DataFrame(columns=df2.columns)
sol_pool, objval =lpex1()

# sol_pool - contains the no of employess assigned for training, relocation etc i.e solution
# odjval - conatains list of total cost for each assignment



if sol_pool:
    
    
    #if solution pool provides three solutions
    
    
    #if cplex solver provides two solutions
    if len(sol_pool)==2:
        
        
        sols,objs=(docplex_modeling(Training_req_count,Relocation_req_count,T_R_req_count,N_T_R_req_count,required_no_of_employees,Total_avialable_candidate,need_extra_solution=1))
        
        # mergining the solution from cplex and docplex
        
        sol_pool=[y for x in [sol_pool, sols] for y in x]  
            
        objval =[y for x in [objval, objs] for y in x]
   

    #if cplex solver solution provides single solution
    else:
        
        #add hiring cost to total cost
        objval[0]=objval[0] + hiring_cost * Need_To_Hire
        
        
            
        #required no of employess are greater than avialable , then there can be only one solution possible
        if Need_To_Hire!=0:
            print('Hire :',Need_To_Hire)
            
            
          
            
        
            
        else:
            
            #if required no of employees are not greater than available, and provide single solution , enhanced the search space for two solution
            
            sols, objs=(docplex_modeling(Training_req_count,Relocation_req_count,T_R_req_count,N_T_R_req_count,required_no_of_employees,Total_avialable_candidate,need_extra_solution=2))
            
            sol_pool=[y for x in [sol_pool, sols] for y in x]
            
            objval =[y for x in [objval, objs] for y in x]
            
            
            #docplex_modeling(Training_req_count,Relocation_req_count,T_R_req_count,N_T_R_req_count,required_no_of_employees,Total_avialable_candidate,need_extra_solution=2)
    
    
    #final solution
    if len(sol_pool)==3:
        for i in range(3):
            Selected_data=pd.DataFrame(columns=df2.columns)
            print('\n\nSolution '+ str(i+1))
            print('Total cost',objval[i])
            for j in range(len(sol_pool[i])):
                value = sol_pool[i][j]
                if value!=0:
                    Selected_data= Selected_data.append(pd.DataFrame(df2[df2['Need']==my_colnames[j]].head(int(value))))
                    print(my_colnames[j]+' : ' + str(value))
                    sol = 'Solution '+ str(i+1)+' .csv'
                    print("Data has been written to solution file in csv format, saved in current working folder")
                    Selected_data.to_csv(sol)    
                
            
            
            print('\n')
        


Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 14000.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 4 columns.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.00 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.00 ticks)
Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1

Populate: phase II 
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 4 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.